<a href="https://colab.research.google.com/github/bahron/machine_learning/blob/main/Front_View_Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Image Clasification - Front View Car**
**Menggunakan Dataset dari kaggle**

Link : https://www.kaggle.com/datasets/sadratmh/front-view-cars

**Mempersiapkan Dataset**

Downloan data dari kaggle

In [ ]:
!wget --no-check-certificate \
    https://github.com/bahron/machine_learning/raw/e1f3cca1764ab1a8f7c457a54017b384bdbe3dfb/frontviewcar.zip \
  -O '/content/front-view-car.zip'

**Ekstrak zip file**

In [ ]:
import os, zipfile

local_zip = '/content/front-view-car.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close


In [ ]:
base_dir = '/content/data'
validation_dir = os.path.join(base_dir, 'val')
train_dir = os.path.join(base_dir, 'train')
bmw_dir = os.path.join(base_dir,'BMW')
ferrari_dir = os.path.join(base_dir, 'Ferrari')
lamborghini_dir = os.path.join(base_dir, 'Lamborghini')

In [ ]:
train_BMW = os.path.join(train_dir, 'BMW')
train_Ferrari = os.path.join(train_dir, 'Ferrari')
train_Lamborghini = os.path.join(train_dir, 'Lamborghini')
val_BMW = os.path.join(validation_dir, 'BMW')
val_Ferrrari = os.path.join(validation_dir, 'Ferrari')
val_Lamborgini = os.path.join(validation_dir, 'Lamborghini')

In [ ]:
os.listdir('/content/data/train')

In [ ]:
os.listdir('/content/data/val')

##**Mempersiapkan image data validasi dan train**

**Mempersiapkan variabel global untuk data training**

In [ ]:
# Parameter input
target = (150, 150)           # mengubah resolusi seluruh gambar menjadi 150x150 piksel
channel = (3, )
input_shape = target + channel
# Batch size
batch_size = 16               # banyaknya citra yang dimasukan dalam setiap steps training
# Epoch
epoch = 10

**Augmentasi image**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1./255)

**Mempersiapkan data train dan validasi melalui fungsi flow()**

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,                            # direktori data latih
        target_size= target,                  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size= batch_size,
        subset='training',
        shuffle=True,  
        class_mode='categorical'              # karena ini merupakan masalah klasifikasi lebih dari 2 kelas, gunakan class_mode = 'categorical'
        )
 
validation_generator = test_datagen.flow_from_directory(
        validation_dir,                       # direktori data validasi
        target_size= target ,                 # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size= batch_size,               
#        subset = 'validation',  
        shuffle = True,
        class_mode='categorical'              # karena ini merupakan masalah klasifikasi lebih dari 2 kelas gunakan class_mode = 'categorical'
        )

Found 59 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [ ]:
train_generator.class_indices

{'BMW': 0, 'Ferrari': 1, 'Lamborghini': 2}

In [ ]:
validation_generator.class_indices

{'BMW': 0, 'Ferrari': 1, 'Lamborghini': 2}

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

model = tf.keras.models.Sequential([                                              #Membentuk model sequential
                                                                                  #Bentuk input adalah gambar dengan ukuran 150x150 dengan 3 byte warna
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)), #Layer Konvolusi pertama
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),                         #Layer Konvolusi kedua
    tf.keras.layers.MaxPooling2D(2,2), 
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),                         #Layer Konvolusi ketiga
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),                                                    #Layer input untuk Deep Learning Neural Network
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(128, activation='relu'),                                #Hiden layer pertama dengan 512 neuron
    
    tf.keras.layers.Dense(512, activation='relu'),                                #Hiden layer kedua dengan 512 neuron
    
    tf.keras.layers.Dense(3, activation='softmax')                                #Layer output
])

model.summary()  

In [ ]:
model.compile(optimizer = 'RMSprop',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 41, # 1312 images = batch_size * steps
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 27, # 876 images = batch_size * steps
    verbose =2,
)